# 1. Importación de paquetes

In [53]:
#Import packages
from modules import preprocess
from modules import training

# 2. Preprocesamiento del *dataset*: [Room Occupancy Estimation Data Set](https://www.kaggle.com/ananthr1/room-occupancy-estimation-data-set)

## Paso 1: Lectura del *dataset* y obtención de datos

In [54]:
df = preprocess.load_csv_data('Occupancy_Estimation.csv')
df.shape

(10129, 19)

In [55]:
df.head()

,Date,Time,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,2017/12/22,10:49:41,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,2017/12/22,10:50:12,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,2017/12/22,10:50:42,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,2017/12/22,10:51:13,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,2017/12/22,10:51:44,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 2: Recodificación de la columna *RoomOccupancyCount*

En este caso, la columna que contiene la variable de salida (*RoomOccupancyCount*) cuenta el número de personas en la sala. Para simplificar, nuestro objetivo va a ser detectar si la sala está vacía o, en cambio, hay alguien (no el número exacto de personas).

Por lo tanto, se **recodificará cualquier valor mayor que 0 en esa columna como 1**, de tal forma que la variable de salida sea binaria:
- 0 -> ausencia.
- 1 -> presencia.

In [56]:
# Show "Room_Occupancy_Count" before processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int64

In [57]:
#Process "Room_Occupancy_Count"
preprocess.recode_dataset_output(df)

# Show "Room_Occupancy_Count" after processing
df["Room_Occupancy_Count"]

0        1
1        1
2        1
3        1
4        1
        ..
10124    0
10125    0
10126    0
10127    0
10128    0
Name: Room_Occupancy_Count, Length: 10129, dtype: int32

## Paso 3: Eliminación de las columnas temporales *Date* y *Time*

Respecto a los datos ofrecidos por las columas *Date* y *Time*, vamos a filtrarlos y dejarlos fuera del proceso. Ya que, aunque los incluyeramos (por ejemplo, juntando ambas partes en una sola columna y convirtiéndolo a formato 'epoch', como long int), los valores de muestras sucesivas de esa columna estarían totalmente correlados entre sí y, como consecuencia, fastidiaríamos a la mayoría de algoritmos que vamos a emplear posteriormente.

Se debe tener en cuenta que no nos estamos olvidando de la información temporal para considerar como si cada valor muestreado (fila de la tabla) fuese independiente de las demás filas. Sabemos que eso no es así, pero estamos considerando que esa información temporal no nos ofrece valor añadido para predecir si la habitación está ocupada o vacía. En caso contrario, se tendrían que usar modelos bastante más complicados para considerar esa relación temporal que indica que, en realidad, las muestras de dos filas adyacentes son consecutivas en el tiempo.

In [58]:
# Filter Date and Time columns
preprocess.remove_time_columns(df)

# Show the result
df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
0,24.94,24.75,24.56,25.38,121,34,53,40,0.08,0.19,0.06,0.06,390,0.769231,0,0,1
1,24.94,24.75,24.56,25.44,121,33,53,40,0.93,0.05,0.06,0.06,390,0.646154,0,0,1
2,25.00,24.75,24.50,25.44,121,34,53,40,0.43,0.11,0.08,0.06,390,0.519231,0,0,1
3,25.00,24.75,24.56,25.44,121,34,53,40,0.41,0.10,0.10,0.09,390,0.388462,0,0,1
4,25.00,24.75,24.56,25.44,121,34,54,40,0.18,0.06,0.06,0.06,390,0.253846,0,0,1


## Paso 4: División del *dataset* en *train* y *test*

In [59]:
# Split dataset into train and test
train_df, test_df = preprocess.split_dataset(df)
train_df.shape, test_df.shape

((7596, 17), (2533, 17))

In [60]:
train_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
7326,25.31,25.31,24.88,25.75,0,0,0,0,0.07,0.05,0.06,0.09,355,0.000000,0,0,0
2407,25.06,25.06,24.44,25.38,5,5,26,18,0.08,0.05,0.06,0.06,360,0.111538,0,0,0
5220,25.13,25.13,24.56,25.44,6,6,32,21,0.07,0.05,0.06,0.06,360,0.000000,0,0,0
4822,25.25,25.25,24.63,25.56,0,0,0,0,0.07,0.05,0.07,0.06,360,0.000000,0,0,0
1121,25.75,25.81,25.38,25.94,0,0,0,0,0.07,0.05,0.06,0.06,715,-2.200000,0,0,0


In [61]:
test_df.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR,Room_Occupancy_Count
8972,25.31,25.31,24.94,25.44,0,0,0,0,0.07,0.05,0.05,0.09,345,-0.103846,0,0,0
4373,25.44,25.44,24.88,25.75,0,0,0,0,0.07,0.05,0.06,0.06,365,-0.119231,0,0,0
2094,25.13,25.19,24.56,25.44,0,0,0,0,0.08,0.05,0.07,0.06,360,0.000000,0,0,0
1478,25.38,25.44,25.00,25.75,0,0,0,0,0.07,0.05,0.07,0.05,380,0.000000,0,0,0
6551,25.44,25.44,25.13,26.00,0,0,0,0,0.08,0.05,0.06,0.10,350,-0.038462,0,0,0


## Paso 5: División del *dataset* de entrenamiento en *features* y *output*

In [62]:
# Get features columns
X = preprocess.get_features(train_df)

# Get output column
y = preprocess.get_output(train_df)

In [63]:
X.head()

,S1_Temp,S2_Temp,S3_Temp,S4_Temp,S1_Light,S2_Light,S3_Light,S4_Light,S1_Sound,S2_Sound,S3_Sound,S4_Sound,S5_CO2,S5_CO2_Slope,S6_PIR,S7_PIR
7326,25.31,25.31,24.88,25.75,0,0,0,0,0.07,0.05,0.06,0.09,355,0.000000,0,0
2407,25.06,25.06,24.44,25.38,5,5,26,18,0.08,0.05,0.06,0.06,360,0.111538,0,0
5220,25.13,25.13,24.56,25.44,6,6,32,21,0.07,0.05,0.06,0.06,360,0.000000,0,0
4822,25.25,25.25,24.63,25.56,0,0,0,0,0.07,0.05,0.07,0.06,360,0.000000,0,0
1121,25.75,25.81,25.38,25.94,0,0,0,0,0.07,0.05,0.06,0.06,715,-2.200000,0,0


In [64]:
y.head()

7326    0
2407    0
5220    0
4822    0
1121    0
Name: Room_Occupancy_Count, dtype: int32

# 2. Entrenamiento y seguimiento de la huella de carbono con **CodeCarbon**

In [65]:
training.train_LR_codecarbon(X,y)
training.train_RF_codecarbon(X,y)
training.train_SVC_codecarbon(X,y)

[codecarbon INFO @ 10:51:59] [setup] RAM Tracking...
[codecarbon INFO @ 10:51:59] [setup] GPU Tracking...
[codecarbon INFO @ 10:51:59] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:51:59] [setup] CPU Tracking...
[codecarbon WARNING @ 10:51:59] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 10:52:01] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:52:01] >>> Tracker's metadata:
[codecarbon INFO @ 10:52:01]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 10:52:01]   Python version: 3.10.4
[codecarbon INFO @ 10:52:01]   Available RAM : 7.935 GB
[codecarbon INFO @ 10:52:01]   CPU count: 6
[codecarbon INFO @ 10:52:01]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:52:01]   GPU count: 1
[codecarbon INFO @ 10:52:01]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 10:52:01] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W


Emissions: 1.3262018201937075e-07 kg


[codecarbon INFO @ 10:52:03] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:52:03] >>> Tracker's metadata:
[codecarbon INFO @ 10:52:03]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 10:52:03]   Python version: 3.10.4
[codecarbon INFO @ 10:52:03]   Available RAM : 7.935 GB
[codecarbon INFO @ 10:52:03]   CPU count: 6
[codecarbon INFO @ 10:52:03]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:52:03]   GPU count: 1
[codecarbon INFO @ 10:52:03]   GPU model: 1 x NVIDIA GeForce GT 710
[codecarbon INFO @ 10:52:03] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codecarbon INFO @ 10:52:03] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 10:52:03] Energy consumed for all CPUs : 0.000002 kWh. All CPUs Power : 32.5 W
[codecarbon INFO @ 10:52:03] 0.000002 kWh of electricity used since the begining.
[codecarbon INFO @ 10:52:03] [setup] 

Emissions: 3.860406235818125e-07 kg


[codecarbon INFO @ 10:52:05] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:52:05] >>> Tracker's metadata:
[codecarbon INFO @ 10:52:05]   Platform system: Windows-10-10.0.19044-SP0
[codecarbon INFO @ 10:52:05]   Python version: 3.10.4
[codecarbon INFO @ 10:52:05]   Available RAM : 7.935 GB
[codecarbon INFO @ 10:52:05]   CPU count: 6
[codecarbon INFO @ 10:52:05]   CPU model: Intel(R) Core(TM) i5-9400F CPU @ 2.90GHz
[codecarbon INFO @ 10:52:05]   GPU count: 1
[codecarbon INFO @ 10:52:05]   GPU model: 1 x NVIDIA GeForce GT 710
C:\Users\INTEL I5\miniconda3\envs\TFG_Project\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
[codecarbon INFO @ 10:52:06] Energy consumed for RAM : 0.000000 kWh. RAM Power : 2.97562837600708 W
[codecarbon INFO @ 10:52:06] Energy consumed for all GPUs : 0.000000 kWh. All GPUs Power : 0.0 W
[codecarbon INFO @ 10:52

Emissions: 7.298640391748753e-08 kg
